In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
%%time
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import json
import xgboost as xgb
import joblib

USERS = "drive/My Drive/VK_Cup_#2/Data/users_prepare.tsv"
HISTORY = "drive/My Drive/VK_Cup_#2/Data/history_prepare.tsv"
VALIDATE = "drive/My Drive/VK_Cup_#2/Data/validate.tsv"
VALIDATE_ANSWERS = "drive/My Drive/VK_Cup_#2/Data/validate_answers.tsv"

PART_DAY = 'drive/My Drive/VK_Cup_#2/Data/part_day_user_ids.json'
PUBLISHERS = 'drive/My Drive/VK_Cup_#2/Data/publisher_user_ids.json'
CITY = "drive/My Drive/VK_Cup_#2/Data/city_user_ids.json"

with open(PART_DAY, "r") as read_file:
    part_day_user_ids = json.load(read_file)
    
with open(PUBLISHERS, "r") as read_file:
    publisher_user_ids = json.load(read_file)

with open(CITY, "r") as read_file:
    cities_user_ids = json.load(read_file)

df_users = pd.read_csv(USERS, sep='\t')
df_validate = pd.read_csv(VALIDATE, sep='\t')
df_history = pd.read_csv(HISTORY, sep='\t')



# Добавление Интервала времени в которое Предположительно показывалось сообщение
df_validate['time_interval'] = df_validate['hour_end'] - df_validate['hour_start']

def string_to_set(strings):
    set_ = set(list(map(int,strings.split(','))))
    return set_
df_validate['publishers'] = df_validate.apply(lambda row: string_to_set(row['publishers']), axis=1)
df_validate['user_ids'] = df_validate.apply(lambda row: string_to_set(row['user_ids']), axis=1)

df_validate['hour_start_time'] = df_validate['hour_start'] % 24
df_validate['hour_end_time'] = df_validate['hour_end'] % 24
df_validate['time_interval_time'] = df_validate['time_interval'] % 24

times = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
def time_diap(hour_start, hour_end):
    if hour_start < hour_end:
        time_interval = set([i for i in range(hour_start, hour_end+1)])
    elif hour_end < hour_start:
        time_interval = set(times - set([i for i in range(hour_end, hour_start+1)]))
    else:
        time_interval = times
    return time_interval
df_validate['time_num_interval'] = df_validate.apply(lambda row: time_diap(row['hour_start_time'], row['hour_end_time']), axis=1)

df_validate['middle_time'] = round((df_validate['hour_end'] + df_validate['hour_start']) / 2)
times = []
for time in np.array(df_validate['middle_time']):
  if time > 240 and time <=720:
    times.append(1)
  elif time > 730 and time <=960:
    times.append(2)
  elif time > 960 and time <=1380:
    times.append(3)
  else:
    times.append(4)

df_validate['part_day'] = times

def people_part_day_platform(time_interval, publishers, user_ids):
    users_who_use_publishers = df_history[(df_history['hour_time'].isin(time_interval)) & (df_history['publisher'].isin(publishers))]['user_id']
    count_partday_people_platform = len(set(users_who_use_publishers) & user_ids)
    return count_partday_people_platform

df_validate['people_partday_platform'] = df_validate.apply(lambda row: people_part_day_platform(row['time_num_interval'], row['publishers'], row['user_ids']), axis=1)

df_validate['percent_people_partday_platform'] = round(df_validate['people_partday_platform'] / df_validate['audience_size'], 2)


list_count_hour_people = []
list_count_hour_people_platform = []

def people_part_day(part_day, user_ids):
    users_who_use_publishers = []
    users_part_day = part_day_user_ids[str(part_day)]
    count_partday_people = len(set(users_part_day) & user_ids)
    return count_partday_people

def people_part_day_platform(part_day, publishers, user_ids):
    users_who_use_publishers = []
    for publisher in publishers:
        users_who_use_publishers += publisher_user_ids[str(publisher)]
    users_part_day = part_day_user_ids[str(part_day)]
    count_partday_people_platform = len(set(users_who_use_publishers) & set(users_part_day) & user_ids)
    return count_partday_people_platform

df_validate['people_patday'] = df_validate.apply(lambda row: people_part_day(row['part_day'], row['user_ids']), axis=1)
df_validate['people_partday_platform'] = df_validate.apply(lambda row: people_part_day_platform(row['part_day'], row['publishers'], row['user_ids']), axis=1)

df_validate['percent_people_patday'] = round(df_validate['people_patday'] / df_validate['audience_size'],2)
df_validate['percent_people_partday_platformy'] = round(df_validate['people_partday_platform'] / df_validate['audience_size'],2)


# Как много людей из целевой категории из разных городов
def counter_unique_cities(user_ids):
    count = len(set(df_users[df_users['user_id'].isin(user_ids)]['city_id']))
    return count
df_validate['count_unique_cities'] = df_validate.apply(lambda row: counter_unique_cities(row['user_ids']), axis=1)


set_all_userid_sex1 = set(df_users[df_users['sex']==1]['user_id'])
set_all_userid_sex2 = set(df_users[df_users['sex']==2]['user_id'])

set_all_userid_age1 = set(df_users[df_users['age']==1]['user_id'])
set_all_userid_age2 = set(df_users[df_users['age']==2]['user_id'])
set_all_userid_age3 = set(df_users[df_users['age']==3]['user_id'])
set_all_userid_age4 = set(df_users[df_users['age']==4]['user_id'])
set_all_userid_age5 = set(df_users[df_users['age']==5]['user_id'])
set_all_userid_age6 = set(df_users[df_users['age']==6]['user_id'])


counts_sex1 = []
counts_sex2 = []

counts_age1 = []
counts_age2 = []
counts_age3 = []
counts_age4 = []
counts_age5 = []
counts_age6 = []

for set_users in df_validate.user_ids:
    
    count_sex1 = len(set_users & set_all_userid_sex1)
    count_sex2 = len(set_users & set_all_userid_sex2)

    count_age1 = len(set_users & set_all_userid_age1)
    count_age2 = len(set_users & set_all_userid_age2)
    count_age3 = len(set_users & set_all_userid_age3)
    count_age4 = len(set_users & set_all_userid_age4)
    count_age5 = len(set_users & set_all_userid_age5)
    count_age6 = len(set_users & set_all_userid_age6)
    
    counts_sex1.append(count_sex1)
    counts_sex2.append(count_sex2)
    
    counts_age1.append(count_age1)
    counts_age2.append(count_age2)
    counts_age3.append(count_age3)
    counts_age4.append(count_age4)
    counts_age5.append(count_age5)
    counts_age6.append(count_age6)
    
df_validate['number_of_sex1'] = counts_sex1
df_validate['number_of_sex2'] = counts_sex2

df_validate['number_of_age1'] = counts_age1
df_validate['number_of_age2'] = counts_age2
df_validate['number_of_age3'] = counts_age3
df_validate['number_of_age4'] = counts_age4
df_validate['number_of_age5'] = counts_age5
df_validate['number_of_age6'] = counts_age6


# Доля мужчин и женщин
df_validate['percentage_sex1'] = round(df_validate['number_of_sex1'] / df_validate['audience_size'],2)
df_validate['percentage_sex2'] = round(df_validate['number_of_sex2'] / df_validate['audience_size'],2)

# Доля людей по возростам
df_validate['percentage_age1'] = round(df_validate['number_of_age1'] / df_validate['audience_size'],2)
df_validate['percentage_age2'] = round(df_validate['number_of_age2'] / df_validate['audience_size'],2)
df_validate['percentage_age3'] = round(df_validate['number_of_age3'] / df_validate['audience_size'],2)
df_validate['percentage_age4'] = round(df_validate['number_of_age4'] / df_validate['audience_size'],2)
df_validate['percentage_age5'] = round(df_validate['number_of_age5'] / df_validate['audience_size'],2)
df_validate['percentage_age6'] = round(df_validate['number_of_age6'] / df_validate['audience_size'],2)


def mean_cpm_in_part_day(part_day, publishers):
    mean_cpm = round(df_history[(df_history['publisher'].isin(publishers)) & (df_history['part_day'] == part_day)]['cpm'].mean())
    return mean_cpm
df_validate['mean_cpm_in_partday_and_publishers'] = df_validate.apply(lambda row: mean_cpm_in_part_day(row['part_day'], row['publishers']), axis=1)


# PUBLISHERS

#Все возможные платформы
all_publishers = np.unique(df_history.publisher)

# Названия столбцов с Publisher'ами
publ_columns = []
name = 'publisher_'
for i in all_publishers:
  publish_name = name + str(i)
  publ_columns.append(publish_name)
publ_columns

# Получение массивов с паблисшерами 0,1
publ = []
number_platform = []
for set_publishers in df_validate.publishers:
  zero_publishers = [0] * len(all_publishers)
  # считаем кол-во платформ
  number = len(set_publishers)
  number_platform.append(number)
  for publisher in set_publishers:
    zero_publishers[int(publisher)-1] = 1
  publ.append(zero_publishers)

#Добавляю кол-во платформ в таблицу
df_validate['number_of_publishers'] = number_platform
# Добавление столбцов в Validate
df_validate = df_validate.join(pd.DataFrame(publ, columns=publ_columns))


# Как много людей из целевой категории из разных городов
def use_that_publishers(publishers, user_ids):
    users_who_use_publishers = []
    for publisher in publishers:
        users_who_use_publishers += publisher_user_ids[str(publisher)]
    count_people = len(user_ids & set(users_who_use_publishers))
    return count_people
df_validate['number_who_use_that_publishers'] = df_validate.apply(lambda row: use_that_publishers(row['publishers'], row['user_ids']), axis=1)

df_validate['percentage_publishers_users'] = round(df_validate['number_who_use_that_publishers'] / df_validate['audience_size'], 2)






df_validate = df_validate.drop(['user_ids', 'publishers', 'time_num_interval', 'hour_start', 'hour_end', 'number_of_sex1', 'part_day', 'number_of_sex2', 'percent_people_partday_platform', 'percentage_sex1', 'percentage_sex2', 'number_who_use_that_publishers', 'count_unique_cities', 'number_of_publishers', 'number_of_age1', 'number_of_age2', 'number_of_age3', 'number_of_age4', 'number_of_age5', 'number_of_age6', 'middle_time'], axis=1)

xgb_model1 = joblib.load('drive/My Drive/VK_Cup_#2/models/xgb_model1_2342.model')
xgb_model2 = joblib.load('drive/My Drive/VK_Cup_#2/models/xgb_model2_2342.model')
xgb_model3 = joblib.load('drive/My Drive/VK_Cup_#2/models/xgb_model3_2342.model')

VALIDATE_ANSWERS = "drive/My Drive/VK_Cup_#2/Data/validate_answers.tsv"
tasks = df_validate.astype('float32')
# Y = pd.read_csv(VALIDATE_ANSWERS, sep='\t')

# Y_1 = Y['at_least_one'].copy()
# Y_2 = Y['at_least_two'].copy()
# Y_3 = Y['at_least_three'].copy()

predictions_1 = xgb_model1.predict(tasks)
predictions_2 = xgb_model2.predict(tasks)
predictions_3 = xgb_model3.predict(tasks)

tasks = tasks.assign(
        at_least_one=predictions_1,
        at_least_two=predictions_2,
        at_least_three=predictions_3,
    )

tasks

CPU times: user 1min 20s, sys: 1.54 s, total: 1min 21s
Wall time: 1min 21s


In [13]:
tasks

,cpm,audience_size,time_interval,hour_start_time,hour_end_time,time_interval_time,people_partday_platform,people_patday,percent_people_patday,percent_people_partday_platformy,percentage_age1,percentage_age2,percentage_age3,percentage_age4,percentage_age5,percentage_age6,mean_cpm_in_partday_and_publishers,publisher_1,publisher_2,publisher_3,publisher_4,publisher_5,publisher_6,publisher_7,publisher_8,publisher_9,publisher_10,publisher_11,publisher_12,publisher_13,publisher_14,publisher_15,publisher_16,publisher_17,publisher_18,publisher_19,publisher_20,publisher_21,percentage_publishers_users,at_least_one,at_least_two,at_least_three
0,220.0,1906.0,95.0,2.0,1.0,23.0,355.0,1457.0,0.76,0.19,0.25,0.41,0.22,0.09,0.01,0.02,257.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.21,0.038631,0.011999,0.006566
1,312.0,1380.0,6.0,23.0,5.0,6.0,163.0,1373.0,0.99,0.12,0.31,0.37,0.21,0.08,0.01,0.03,204.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.12,0.009856,0.000132,0.000335
2,70.0,888.0,20.0,5.0,1.0,20.0,607.0,661.0,0.74,0.68,0.26,0.37,0.21,0.12,0.01,0.02,194.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.82,0.082200,0.009806,0.000800
3,240.0,440.0,82.0,23.0,9.0,10.0,203.0,334.0,0.76,0.46,0.27,0.42,0.18,0.10,0.01,0.02,186.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.54,0.230618,0.128600,0.070380
4,262.0,1476.0,238.0,8.0,6.0,22.0,768.0,899.0,0.61,0.52,0.00,0.00,0.15,0.85,0.00,0.00,180.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.73,0.397324,0.280375,0.225929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,127.0,368.0,59.0,15.0,2.0,11.0,113.0,270.0,0.73,0.31,0.26,0.36,0.24,0.11,0.01,0.02,222.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.38,0.070865,0.036810,0.018205
1004,90.0,484.0,4.0,20.0,0.0,4.0,100.0,354.0,0.73,0.21,0.30,0.32,0.21,0.12,0.03,0.01,216.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.23,0.011595,0.000054,0.000282
1005,122.0,704.0,5.0,2.0,7.0,5.0,197.0,701.0,1.00,0.28,0.29,0.41,0.19,0.07,0.01,0.03,215.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.28,0.011326,0.000122,0.000349
1006,138.0,1210.0,237.0,20.0,17.0,21.0,793.0,873.0,0.72,0.66,0.00,0.00,0.00,0.08,0.40,0.52,198.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.80,0.369857,0.264408,0.206632


In [0]:
xgb_model1 = joblib.load('drive/My Drive/VK_Cup_#2/models/xgb_model1_316.model')
xgb_model2 = joblib.load('drive/My Drive/VK_Cup_#2/models/xgb_model2_316.model')
xgb_model3 = joblib.load('drive/My Drive/VK_Cup_#2/models/xgb_model3_316.model')

In [5]:
predictions_1 = xgb_model1.predict(tasks)
predictions_2 = xgb_model2.predict(tasks)
predictions_3 = xgb_model3.predict(tasks)

NameError: ignored